## Setup Jupyter Notebook

In [1]:
from pathlib import Path
import sys

notebook_directory_parent = Path.cwd().resolve().parent.parent
if str(notebook_directory_parent) not in sys.path:
    sys.path.append(str(notebook_directory_parent))

In [2]:
%matplotlib inline

import numpy as np
import scipy
import sympy
from numpy import linspace
from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt

In [3]:
from T1000.numerical.RungeKuttaMethod import RungeKuttaMethod
from T1000.numerical.RKMethods.bCoefficients import bCoefficients
from T1000.numerical.RKMethods.DOPRI5Coefficients import DOPRI5Coefficients

cf. [Examples for solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp)

In [4]:
def exponential_decay(t, y):
    return -0.5 * y

In [5]:
sol = solve_ivp(exponential_decay, [0, 10], [2, 4, 8])
print("t\n", sol.t)
print("y\n", sol.y)

t
 [ 0.          0.11487653  1.26364188  3.06061781  4.81611105  6.57445806
  8.33328988 10.        ]
y
 [[2.         1.88836035 1.06327177 0.43319312 0.18017253 0.07483045
  0.03107158 0.01350781]
 [4.         3.7767207  2.12654355 0.86638624 0.36034507 0.14966091
  0.06214316 0.02701561]
 [8.         7.5534414  4.25308709 1.73277247 0.72069014 0.29932181
  0.12428631 0.05403123]]


# An example for unit tests

cf. [Runge-Kutta method from OK state](https://math.okstate.edu/people/yqwang/teaching/math4513_fall11/Notes/rungekutta.pdf)

Depending on one's notation, for "stage" $s=4$, or $m=4$, then for

$\alpha_2 = \frac{1}{2}, \, \alpha_3 = \frac{1}{2}, \, \alpha_4 = 1 \equiv \\
c_2 = \frac{1}{2}, \, c_3 = \frac{1}{2}, \, c_4 = 1$

$\beta_{21} = \frac{1}{2}, \, \beta_{31} = 0, \, \beta_{32} = \frac{1}{2}, \, \beta_{41} = \beta_{42} = 0, \, \beta_{43} = 1 \equiv \\
a_{21} = \frac{1}{2}, \, a_{31} =0, \, a_{32} = \frac{1}{2}, \, a_{41} = a_{42} = 0, \, a_{43} = 1$

$c_1 = c_4 = \frac{1}{6}, \, c_2 = c_3 = \frac{1}{3} \equiv b_1 = b_4 = \frac{1}{6}, \, b_2 = b_3 = \frac{1}{3}$ 

(authors having different notation sucks).


In [5]:
cs_for_rk4 = [0.5, 0.5, 1]
as_for_rk4 = [0.5, 0, 0.5, 0, 0, 1]
bs_for_rk4 = [1/6., 1/3., 1/3., 1/6.]
rk4 = RungeKuttaMethod(4, cs_for_rk4, as_for_rk4, bs_for_rk4)

Consider example

$\begin{cases}
    & y'  =y -x^2 + 1\\
    & y(0) = 0.5
\end{cases}$


Its exact solution is 
$ y= y(x) = x^2 + 2x + 1 - \frac{1}{2} \exp{x}$

In [9]:
def derivative(x, y):
    return y - np.power(x, 2) + 1.0

def exact_y(x):
    return np.power(x, 2) + 2.0 * x + 1.0 - 0.5 * np.exp(x)

1. **We first solve this problem using RK4 with** $h = 0.5$ and from $t=0$ to $t=2$.

With step size $h = 0.5$, it takes 4 steps: $t_0 = 0, \, t_1 = 0.5, \, t_2 = 1, \, t_3 = 1.5, \, t_4 = 2$

*Step 0* \quad \, $x_0 = 0, \, y_0 = 0.5$ 

*Step 1* \quad \, $x_1 = 0.5, \, h = 0.5$

In [12]:
ks0 = rk4._calculate_k_coefficients(0.5, 0.0, 0.5, derivative)
print(ks0)
y1 = rk4.calculate_next_step(0.5, 0.0, 0.5, derivative)
print(y1)
print(exact_y(0))
print(exact_y(0.5))

[1.5, 1.8125, 1.890625, 2.1953125]
1.4251302083333333
0.5
1.425639364649936


*Step 2* $\quad \, x_2 = 1$

In [13]:
ks1 = rk4._calculate_k_coefficients(y1, 0.5, 0.5, derivative)
print(ks1)
y2 = rk4.calculate_next_step(y1, 0.5, 0.5, derivative)
print(y2)
print(exact_y(1.))

[2.175130208333333, 2.4064127604166665, 2.4642333984375, 2.657246907552083]
2.6396026611328125
2.6408590857704777


## Calculated values for unit tests

In [6]:
alpha_for_rk4 = [0.5, 0.5, 1.]
beta_for_rk4 = [0.5, 0., 0.5, 0., 0., 1.]
c_for_rk4 = [1./6., 1./3., 1./3., 1./6.]
rk4 = RungeKuttaMethod(4, alpha_for_rk4, beta_for_rk4, c_for_rk4)

In [7]:
m = 4
print(rk4._beta_coefficients)
for i in range(2, m + 1):
    for j in range(1, i):
        print(i, " ", j, " ", rk4.get_beta_ij(i, j))

[0.5, 0.0, 0.5, 0.0, 0.0, 1.0]
2   1   0.5
3   1   0.0
3   2   0.5
4   1   0.0
4   2   0.0
4   3   1.0


In [8]:
print(rk4._alpha_coefficients)
for i in range(2, m + 1):
    print(i, " ", rk4.get_alpha_i(i))

[0.5, 0.5, 1.0]
2   0.5
3   0.5
4   1.0


In [9]:
print(rk4._c_coefficients)
for i in range(1, m + 1):
    print(i, " ", rk4.get_c_i(i))

[0.16666666666666666, 0.3333333333333333, 0.3333333333333333, 0.16666666666666666]
1   0.16666666666666666
2   0.3333333333333333
3   0.3333333333333333
4   0.16666666666666666


In [14]:
x_n = [np.array([2, 4, 8]), np.array([1.88836035, 3.7767207, 7.5534414])]
t_n = [0., 0.11487653, 1.26364188]

In [15]:
kresult = rk4._calculate_k_coefficients(x_n[0], t_n[0], t_n[1] - t_n[0], exponential_decay)
print(kresult)

[array([-1., -2., -4.]), array([-0.97128087, -1.94256173, -3.88512347]), array([-0.97210566, -1.94421131, -3.88842262]), array([-0.94416394, -1.88832788, -3.77665575])]


In [17]:
result1 = rk4.calculate_next_step(x_n[0], t_n[0], t_n[1] - t_n[0], exponential_decay)
print(result1)

[1.88836037 3.77672073 7.55344146]


In [16]:
result2 = rk4.calculate_next_step(x_n[1], t_n[1], t_n[2] - t_n[1], exponential_decay)
print(result2)

[1.06414256 2.12828513 4.25657025]


In [15]:
[1, 2, 3] + [ 4, 5, 6]

[1, 2, 3, 4, 5, 6]

In [17]:
np.array([2])

array([2])

# Embedded Formulas of Order 5

cf. pp. 178, Table 5.2. Dormand-Prince 5(4) (DOPRI5) **Ordinary Differential Equations, Vol. 1. Nonstiff-Problems**

In [5]:
# print(DOPRI5Coefficients.b_coefficients.get_ith_element(1))
deltas = []
for i in range(1, 8):
    delta = \
        DOPRI5Coefficients.b_coefficients.get_ith_element(i) - \
            DOPRI5Coefficients.bstar_coefficients.get_ith_element(i)
    deltas.append(delta)
    if (type(delta) != int):
        print(delta.simplify())
    else:
        print(delta)

71/57600
0
-71/16695
71/1920
-17253/339200
22/525
-1/40


# Coefficients for dense output

cf. https://math.stackexchange.com/questions/2947231/how-can-i-derive-the-dense-output-of-ode45

In [7]:
bs = DOPRI5Coefficients.b_coefficients
ds = DOPRI5Coefficients.dense_output_coefficients
cstars = DOPRI5Coefficients.cstar_coefficients

In [11]:
-2 * (1 + 4 * bs.get_ith_element(1) - 4 * cstars.get_ith_element(1))

-12715105075/11282082432

# Hermite Interpolation

In [9]:
from sympy import Matrix, Symbol, symbols, pprint

In [7]:
theta, y_n, y_np1, f_n, f_np1, h = symbols("theta y_n y_np1 f_n f_np1 h")

cf. pp. 916 17.2.2 Dense Output, Ch. 17. Integration of Ordinary Differential Equations of **Numerical Recipes**, 3rd Ed., and  

In [14]:
hermite_interpolation = (1 - theta) * y_n + theta * y_np1 + \
  theta * (theta - 1) * ((1 - 2 * theta) * (y_np1 - y_n) + (theta - 1) * h *f_n + theta * h * f_np1)
pprint(hermite_interpolation)

θ⋅yₙₚ₁ + θ⋅(θ - 1)⋅(fₙ⋅h⋅(θ - 1) + fₙₚ₁⋅h⋅θ + (1 - 2⋅θ)⋅(-yₙ + yₙₚ₁)) + yₙ⋅(1 
- θ)


In [17]:
pprint(hermite_interpolation.expand())
pprint(sympy.collect(hermite_interpolation.expand(), y_n, evaluate=False)[y_n])
pprint(sympy.collect(hermite_interpolation.expand(), y_np1, evaluate=False)[y_np1])
pprint(sympy.collect(hermite_interpolation.expand(), f_n, evaluate=False)[f_n])
pprint(sympy.collect(hermite_interpolation.expand(), f_np1, evaluate=False)[f_np1])

      3           2                    3           2      3         3         
fₙ⋅h⋅θ  - 2⋅fₙ⋅h⋅θ  + fₙ⋅h⋅θ + fₙₚ₁⋅h⋅θ  - fₙₚ₁⋅h⋅θ  + 2⋅θ ⋅yₙ - 2⋅θ ⋅yₙₚ₁ - 3

  2         2          
⋅θ ⋅yₙ + 3⋅θ ⋅yₙₚ₁ + yₙ
   3      2    
2⋅θ  - 3⋅θ  + 1
     3      2
- 2⋅θ  + 3⋅θ 
   3        2      
h⋅θ  - 2⋅h⋅θ  + h⋅θ
   3      2
h⋅θ  - h⋅θ 
